# Pluto / Eris SWU parameters


This notebooks computes the Z parameter for the Shallue-van de Woestijne Map in the Pluto/ Eris cycle of curves.

In [19]:
# Pluto Base Field
p = 0x24000000000024000130e0000d7f70e4a803ca76f439266f443f9a5cda8a6c7be4a7a5fe8fadffd6a2a7e8c30006b9459ffffcd300000001

# Eris Base Field
q = 0x24000000000024000130e0000d7f70e4a803ca76f439266f443f9a5c7a8a6c7be4a775fe8e177fd69ca7e85d60050af41ffffcd300000001

a = 0
b = 57

# Pluto is y^2 = x^3 + aX + B

In [20]:
Fp = GF(p)
Fq = GF(q)

In [21]:
# https://www.rfc-editor.org/rfc/rfc9380.html#svdw-z-code
# Arguments:
# - F, a field object, e.g., F = GF(2^521 - 1)
# - A and B, the coefficients of the curve y^2 = x^3 + A * x + B
def find_z_svdw(F, A, B, init_ctr=1):
    g = lambda x: F(x)^3 + F(A) * F(x) + F(B)
    h = lambda Z: -(F(3) * Z^2 + F(4) * A) / (F(4) * g(Z))
    # NOTE: if init_ctr=1 fails to find Z, try setting it to F.gen()
    ctr = init_ctr
    while True:
        for Z_cand in (F(ctr), F(-ctr)):
            # Criterion 1:
            #   g(Z) != 0 in F.
            if g(Z_cand) == F(0):
                continue
            # Criterion 2:
            #   -(3 * Z^2 + 4 * A) / (4 * g(Z)) != 0 in F.
            if h(Z_cand) == F(0):
                continue
            # Criterion 3:
            #   -(3 * Z^2 + 4 * A) / (4 * g(Z)) is square in F.
            if not is_square(h(Z_cand)):
                continue
            # Criterion 4:
            #   At least one of g(Z) and g(-Z / 2) is square in F.
            if is_square(g(Z_cand)) or is_square(g(-Z_cand / F(2))):
                return Z_cand
        ctr += 1

In [22]:
# Helper functions
def u_64_little_endian(n):
    str_hex = hex(n)
    str_hex_without_0x = str_hex[2:]
    full_width_str = '0' * (112 - len(str_hex_without_0x)) + str_hex_without_0x
    assert len(full_width_str) == 112

    res = []
    for i in range(7):
        temp = '0x' + full_width_str[112 - 16 * (i + 1) : 112 - 16 * i]
        res.append(temp)
    return res

def print_for_rust(elem):
    print( hex(elem))
    print('\n')
    print('[')
    for limb in u_64_little_endian(elem):
        print(limb + ",")
    print(']')
    print('------\n')

In [23]:
pluto_z = find_z_svdw(Fp, a, b)
print_for_rust(pluto_z)


0x1


[
0x0000000000000001,
0x0000000000000000,
0x0000000000000000,
0x0000000000000000,
0x0000000000000000,
0x0000000000000000,
0x0000000000000000,
]
------



In [24]:
eris_z = find_z_svdw(Fq, a, b)
print_for_rust(eris_z)

0x1


[
0x0000000000000001,
0x0000000000000000,
0x0000000000000000,
0x0000000000000000,
0x0000000000000000,
0x0000000000000000,
0x0000000000000000,
]
------

